In [23]:
from os import environ
from typing import List, Dict, Iterable, Union, Any

# Load env variables and create client
from anthropic import AnthropicVertex
from anthropic.types import Message, ToolParam, ToolResultBlockParam, ToolUseBlock

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = AnthropicVertex(region=region, project_id=project_id)
model = "claude-sonnet-4-5@20250929"

In [24]:
# Helper functions
def add_user_message(messages: List[dict], msg: Any):
    """Adds a user message to the messages list."""
    user_message = {
        "role": "user",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(user_message)


def add_assistant_message(messages: List[dict], msg: Union[Message, dict, str]):
    """Adds an assistant message to the messages list."""
    assistant_message = {
        "role": "assistant",
        "content": msg.content if isinstance(msg, Message) else msg,
    }
    messages.append(assistant_message)


def chat(
        messages: List[dict],
        system: str = None,
        temperature: float = 1.0,
        stop_sequences: Iterable[str] = [],
        tools: Iterable[ToolParam] = None,
        tool_choice: Dict = None,
):
    """Sends a chat request to the Claude model with the given messages."""
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system is not None:
        params["system"] = system

    if tools is not None:
        params["tools"] = tools

    if tool_choice is not None:
        params["tool_choice"] = tool_choice

    message = client.messages.create(**params)
    return message


def text_from_message(message: Message) -> Iterable[str]:
    """Extracts and concatenates all text blocks from a Message object."""
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [25]:
# Tools and Schemas

from datetime import datetime, timedelta
from enum import Enum


class TimeUnit(str, Enum):
    """Enum for time duration units."""

    SECONDS = "seconds"
    MINUTES = "minutes"
    HOURS = "hours"
    DAYS = "days"
    WEEKS = "weeks"
    MONTHS = "months"
    YEARS = "years"


def add_duration_to_datetime(
        datetime_str: str,
        duration: int = 0,
        unit: TimeUnit = TimeUnit.DAYS,
        input_format: str = "%Y-%m-%d",
):
    date = datetime.strptime(datetime_str, input_format)

    # Convert string to enum if needed (for backwards compatibility)
    if isinstance(unit, str):
        unit = TimeUnit(unit)

    if unit == TimeUnit.SECONDS:
        new_date = date + timedelta(seconds=duration)
    elif unit == TimeUnit.MINUTES:
        new_date = date + timedelta(minutes=duration)
    elif unit == TimeUnit.HOURS:
        new_date = date + timedelta(hours=duration)
    elif unit == TimeUnit.DAYS:
        new_date = date + timedelta(days=duration)
    elif unit == TimeUnit.WEEKS:
        new_date = date + timedelta(weeks=duration)
    elif unit == TimeUnit.MONTHS:
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == TimeUnit.YEARS:
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(f"----\nSetting the following reminder for {timestamp}:\n{content}\n----")


add_duration_to_datetime_schema: ToolParam = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "enum": [
                    "seconds",
                    "minutes",
                    "hours",
                    "days",
                    "weeks",
                    "months",
                    "years",
                ],
                "description": "The unit of time for the duration. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

set_reminder_schema: ToolParam = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}

batch_tool_schema: ToolParam = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}

article_summary_schema: ToolParam = {
    "name": "article_summary",
    "description": "Extracts structured data from articles",
    "input_schema": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "author": {"type": "string"},
            "key_insights": {
                "type": "array",
                "items": {"type": "string"}
            }
        }
    }
}

pass

In [26]:
def get_current_datetime(datetime_format="%Y-%m-%d %H:%M:%S"):
    if not datetime_format:
        raise ValueError("datetime_format must be provided")
    now = datetime.now()
    return now.strftime(datetime_format)


get_current_datetime_schema: ToolParam = {
    "name": "get_current_datetime",
    "description": "Returns the current date and time as a formatted string. Retrieves the current system datetime and formats it according to the specified format string using Python's strftime directives.",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_format": {
                "type": "string",
                "description": "The format string for datetime output using Python strftime directives. Common formats: '%Y-%m-%d %H:%M:%S' (2025-12-30 14:30:45), '%Y-%m-%d' (2025-12-30), '%H:%M:%S' (14:30:45), '%B %d, %Y' (December 30, 2025), '%Y-%m-%dT%H:%M:%SZ' (ISO 8601). See Python strftime documentation for all directives.",
                "default": "%Y-%m-%d %H:%M:%S",
            }
        },
        "required": [],
    },
}

get_current_datetime()
get_current_datetime("%B %d, %Y")

'December 31, 2025'

In [27]:
messages: Iterable[Dict] = []
messages.append(
    {"role": "user", "content": "What is the exact time, formatted as HH:MM:SS?"}
)

response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema],
)
# Message can contain
messages.append({"role": "assistant", "content": response.content})
tool_result = get_current_datetime(**response.content[-1].input)
response

Message(id='msg_vrtx_01JekWB9csGytNUm3zHwWGi6', content=[ToolUseBlock(id='toolu_vrtx_01KixhPcsMzsKzbUo5Vfh6KH', input={'datetime_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=742, output_tokens=62, server_tool_use=None, service_tier=None))

In [28]:
## Tool result block from user message
messages.append(
    {
        "role": "user",
        "content": [
            # tool result block
            {
                "type": "tool_result",
                "tool_use_id": response.content[-1].id,
                "content": tool_result,
                "is_error": False,
            }
        ],
    }
)

In [29]:
response = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema],
)
response.content[0].text

'The exact time is **15:06:54**.'

# Run conversation

In [30]:
from json import dumps, loads


def run_batch(invocations: List[Dict]) -> List[Dict]:
    """Executes a batch of tool calls and returns their results."""
    batch_output = []
    for invocation in invocations:
        tool_name = invocation["name"]
        tool_input = loads(invocation["arguments"])
        result = run_tool(tool_name, tool_input)
        batch_output.append({"tool_name": tool_name, "output": result})
    return batch_output


def run_tool(tool_name: str, tool_input: Dict) -> Union[str, None, List[Dict]]:
    """Executes a single tool call and returns the result as a ToolResultBlockParam."""
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    elif tool_name == "batch_tool":
        return run_batch(tool_input["invocations"])
    elif tool_name == "article_summary":
        return None
    else:
        raise NotImplementedError(f"Tool '{tool_name}' is not implemented.")


def run_tools(message: Message) -> List[ToolResultBlockParam]:
    """Executes tool calls found in the assistant's message content."""
    tool_results: List[ToolResultBlockParam] = []
    for block in message.content:
        if isinstance(block, ToolUseBlock):
            tool_name = block.name
            tool_input = block.input
            try:
                result = run_tool(tool_name, tool_input)
                print(
                    f"Tool '{tool_name}' called with input {tool_input}, result: {result}"
                )
                tool_results.append(
                    ToolResultBlockParam(
                        tool_use_id=block.id,
                        content=dumps(result),
                        is_error=False,
                        type="tool_result",
                    )
                )
            except Exception as e:
                print(f"Error executing tool '{tool_name}': {e}")
                tool_results.append(
                    ToolResultBlockParam(
                        tool_use_id=block.id,
                        content=f" Error: {e}",
                        is_error=True,
                        type="tool_result",
                    )
                )

    return tool_results

In [31]:
def run_conversation(messages: List[Dict] = None) -> List[Dict]:
    """Runs a conversation loop with tool usage until completion."""
    if messages is None:
        messages = []
    while True:
        response = chat(
            messages=messages,
            tools=[
                get_current_datetime_schema,
                add_duration_to_datetime_schema,
                set_reminder_schema,
                batch_tool_schema,
                article_summary_schema,
            ],
        )
        add_assistant_message(messages, response)
        print("Assistant:", text_from_message(response))
        if response.stop_reason != "tool_use":
            break
        tool_results = run_tools(response)
        add_user_message(messages, tool_results)
    return messages

In [32]:
messages: List[Dict] = []
add_user_message(
    messages,
    "What is the current date and time formatted as 'Month Day, Year HH:MM:SS'? Also what is the current time in ISO 8601 format?",
)
messages = run_conversation(messages)

Assistant: I'll get the current date and time in both formats for you.
Tool 'get_current_datetime' called with input {'datetime_format': '%B %d, %Y %H:%M:%S'}, result: December 31, 2025 15:06:58
Tool 'get_current_datetime' called with input {'datetime_format': '%Y-%m-%dT%H:%M:%S'}, result: 2025-12-31T15:06:58
Assistant: The current date and time is:

- **'Month Day, Year HH:MM:SS' format**: December 31, 2025 15:06:58
- **ISO 8601 format**: 2025-12-31T15:06:58


In [33]:
messages: List[Dict] = []
add_user_message(
    messages,
    "Set a reminder for my doctor's appointment on it's 177 days after January 1, 2025 at 10:00 AM.",
)
messages = run_conversation(messages)

Assistant: I'll help you set a reminder for your doctor's appointment. First, let me calculate what date is 177 days after January 1, 2025, and then set the reminder for 10:00 AM on that date.
Tool 'add_duration_to_datetime' called with input {'datetime_str': '2025-01-01', 'duration': 177, 'unit': 'days', 'input_format': '%Y-%m-%d'}, result: Friday, June 27, 2025 12:00:00 AM
Assistant: Now I'll set the reminder for June 27, 2025 at 10:00 AM:
----
Setting the following reminder for 2025-06-27T10:00:00:
Doctor's appointment
----
Tool 'set_reminder' called with input {'content': "Doctor's appointment", 'timestamp': '2025-06-27T10:00:00'}, result: None
Assistant: Perfect! I've set a reminder for your doctor's appointment on **Friday, June 27, 2025 at 10:00 AM** (which is 177 days after January 1, 2025). You'll receive a notification at that time.


In [34]:
messages: List[Dict] = []
add_user_message(
    messages,
    "Set two reminders at 10am tomorrow, for my doctor's appointment and another for my taxes.",
)
messages = run_conversation(messages)

Assistant: I'll set those two reminders for you at 10am tomorrow. First, let me get tomorrow's date and then create both reminders.
Tool 'get_current_datetime' called with input {'datetime_format': '%Y-%m-%d'}, result: 2025-12-31
Assistant: 
Tool 'add_duration_to_datetime' called with input {'datetime_str': '2025-12-31', 'duration': 1, 'unit': 'days', 'input_format': '%Y-%m-%d'}, result: Thursday, January 01, 2026 12:00:00 AM
Assistant: 
----
Setting the following reminder for 2026-01-01T10:00:00:
Doctor's appointment
----
----
Setting the following reminder for 2026-01-01T10:00:00:
Taxes
----
Tool 'batch_tool' called with input {'invocations': [{'name': 'set_reminder', 'arguments': '{"content": "Doctor\'s appointment", "timestamp": "2026-01-01T10:00:00"}'}, {'name': 'set_reminder', 'arguments': '{"content": "Taxes", "timestamp": "2026-01-01T10:00:00"}'}]}, result: [{'tool_name': 'set_reminder', 'output': None}, {'tool_name': 'set_reminder', 'output': None}]
Assistant: Perfect! I've se

## Tools for Structured Data
The tool-based approach works by creating a JSON schema that defines the exact structure of data you want to extract. Instead of hoping Claude formats its response correctly, you're essentially giving Claude a function to call with specific parameters that match your desired output structure.

In [36]:
messages: List[Dict] = []
add_user_message(
    messages,
    """
    Write a one paragraph article about computer science include a title and author name. Use the article_summary tool to extract the title, author, and 3 key insights from the article.
    """,
)
messages = run_conversation(messages)

Assistant: I'll write a one paragraph article about computer science and then extract its key information using the article_summary tool.

**Article:**

**Title:** The Transformative Power of Computer Science in Modern Society

**Author:** Dr. Sarah Mitchell

Computer science has emerged as one of the most influential disciplines of the 21st century, fundamentally reshaping how we live, work, and communicate. From the algorithms that power social media platforms and search engines to the artificial intelligence systems revolutionizing healthcare diagnostics and autonomous vehicles, computer science principles underpin nearly every aspect of modern technological infrastructure. The field encompasses diverse areas including software engineering, data science, cybersecurity, and machine learning, each contributing uniquely to solving complex real-world problems. As we generate unprecedented amounts of data daily, computer scientists develop sophisticated methods to process, analyze, and e